In [117]:
import networkx as nx
import numpy as np
import random

In [118]:
def shortest_way_fan(g,s,t):
  try:
    path=nx.astar_path(g,s,t,weight='cost')
    edges=[[path[i],path[i+1]] for i in range(len(path)-1)]
    return edges
  except Exception:
    return False

In [119]:
def residual(u,f):
  return u-f+np.transpose(f)

In [120]:
b = np.array([15, 0, -10, 10, 0, -15])

u = np.array([
    [0, 15, 0, 0, 5, 0],
    [0, 0, 15, 0, 0, 0],
    [0, 0, 0, 0, 0, 10],
    [0, 0, 0, 0, 8, 10],
    [0, 0, 4, 0, 0, 8],
    [0, 0, 0, 0, 0, 0] ])

c = np.array([
    [0,5,0,0,-15,0],
    [0,0,5,0,0,0],
    [0,0,0,0,0,5],
    [0,0,0,0,0,6],
    [0,0,3,0,0,5],
    [0,0,0,0,0,0]])

c1 = {(0, 1):5, (0, 4):-15, (1, 2):5, (2, 5):5, (3,4):0 ,(3, 5):6, (4, 2):3, (4, 5):5}

In [121]:
def residual_cost_2(uf, c):
      n = len(uf)
      c2 = {}

      for i in range(n):
          for j in range(n):
              if uf[i][j]>0:
                  if (i,j) in c:
                      c2[i,j] = c[i,j]
                  else:
                      c2[i,j] = -c[j,i]

      return c2

In [122]:
def min_cost(u,b,c1):
        f = np.full((len(u),len(u)), 0)
        uf = residual(u, f)
        gf = nx.from_numpy_array(uf, create_using = nx.DiGraph)
        c1 = residual_cost_2(uf, c1)
        for (start, end) in c1.keys():
          if (start, end) in gf.edges:
            gf.edges[start, end]['cost'] = c1[start, end]
        b2=b.copy()
        array_for_s = [i for i in range (len(u)) if b2[i]>0]
        array_for_t = [i for i in range (len(u)) if b2[i]<0]
        for s1 in array_for_s:
          for t1 in array_for_t:
            if shortest_way_fan(gf,s1,t1) != False:
              s,t = s1,t1
              break
        shortest_way = shortest_way_fan(gf,s,t)
        while shortest_way != False:
            if len([x for x in b2 if (x > 0 or x < 0)]) == 0:
              break
            else:
              min_bw = min(uf[i][j] for i,j in shortest_way)
              gamma = min(min_bw, b2[s], -b2[t])

              for i,j in shortest_way:
                delta = min(u[i][j]-f[i][j],gamma)
                f[i][j] += delta
                f[j][i] -= gamma - delta


              b2[s]-=gamma
              b2[t]+=gamma


              if b2[s] <= 0:
                  array_for_s.remove(s)
              if b2[t] >= 0:
                  array_for_t.remove(t)

              uf = residual(u, f)
              gf = nx.from_numpy_array(uf, create_using = nx.DiGraph)
              c1 = residual_cost_2(uf, c1)
              for (start, end) in c1.keys():
                if (start, end) in gf.edges:
                  gf.edges[start, end]['cost'] = c1[start, end]
              for s1 in array_for_s:
                for t1 in array_for_t:
                  if (shortest_way_fan(gf,s1,t1)!= False):
                    s,t = s1,t1
                    break
              shortest_way = shortest_way_fan(gf,s,t)
              print(s,t,shortest_way_fan(gf,s,t))
        return f

In [123]:
f=min_cost(u,b,c1)
f

3 5 [[3, 4], [4, 5]]
3 5 [[3, 5]]
0 2 [[0, 1], [1, 2]]
0 5 [[0, 4], [4, 5]]
0 5 [[0, 4], [4, 3], [3, 5]]
0 5 [[0, 1], [1, 2], [2, 4], [4, 3], [3, 5]]
0 5 [[0, 1], [1, 2], [2, 5]]


array([[ 0, 10,  0,  0,  5,  0],
       [ 0,  0, 10,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  3,  7],
       [ 0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  0,  0]])

In [124]:
cost=0
k=len(u)
for i in range(k):
  for j in range(k):
    cost+=f[i,j] * c[i,j]
cost

107